### Spark Exercises

Within your `codeup-data-science` directory, create a new repo named `spark-exercises`. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

Save this work in your `spark-exercises` repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named `spark101` for this exercise.

In [1]:
import pyspark
import pandas as pd
import numpy as np

from pyspark.sql.functions import col, expr
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.sql.functions import when
from pyspark.sql.functions import asc, desc
from pyspark.sql.functions import month, year, quarter
from pyspark.sql.functions import *



1. Create a spark data frame that contains your favorite programming languages.

    - The name of the column should be `language`
    - View the schema of the dataframe
    - Output the shape of the dataframe
    - Show the first 5 records in the dataframe

In [3]:
np.random.seed(456)

pl_df = pd.DataFrame(np.random.choice(['python', 'sql', 'java', 'c', 'r'], 20))
pl_df.columns = ['language']
spark = pyspark.sql.SparkSession.builder.getOrCreate()
df = spark.createDataFrame(pl_df)
df

DataFrame[language: string]

In [4]:
df.printSchema()

root
 |-- language: string (nullable = true)



In [7]:
print('DataFrame shape: ', df.count(), ' x ', len(df.columns))

DataFrame shape:  20  x  1


In [8]:
df.show(5)

+--------+
|language|
+--------+
|       c|
|       c|
|     sql|
|    java|
|       r|
+--------+
only showing top 5 rows



2. Load the `mpg` dataset as a spark dataframe.

    a. For each vehicle, create 1 column of output that contains a message like the one below:
    
   > - The 1999 audi a4 has a 4 cylinder engine.
  

In [9]:
from pydataset import data

mpg = spark.createDataFrame(data('mpg'))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [10]:
desc_column = mpg.select(
    concat(lit('The '), 
    mpg.year, lit(' '), 
    mpg.manufacturer, 
    lit(' '), 
    mpg.model, 
    lit(' has a '), 
    mpg.cyl, 
    lit(' cylinder engine.')).alias('description'))

In [11]:
desc_column.show(5, truncate = False)

+-----------------------------------------+
|description                              |
+-----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 6 cylinder engine.|
+-----------------------------------------+
only showing top 5 rows



 b. Transform the `trans` column so that it only contains either `manual` or `auto`.

In [12]:
mpg.select('trans', regexp_replace('trans', r"\(.+$", "")).show()

+----------+---------------------------------+
|     trans|regexp_replace(trans, \(.+$, , 1)|
+----------+---------------------------------+
|  auto(l5)|                             auto|
|manual(m5)|                           manual|
|manual(m6)|                           manual|
|  auto(av)|                             auto|
|  auto(l5)|                             auto|
|manual(m5)|                           manual|
|  auto(av)|                             auto|
|manual(m5)|                           manual|
|  auto(l5)|                             auto|
|manual(m6)|                           manual|
|  auto(s6)|                             auto|
|  auto(l5)|                             auto|
|manual(m5)|                           manual|
|  auto(s6)|                             auto|
|manual(m6)|                           manual|
|  auto(l5)|                             auto|
|  auto(s6)|                             auto|
|  auto(s6)|                             auto|
|  auto(l4)| 

3. Load the `tips` dataset as a spark dataframe.

 a. What percentage of observations are smokers?

In [13]:
from pydataset import data

tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [14]:
print('DataFrame shape: ', tips.count(), ' x ', len(tips.columns))

DataFrame shape:  244  x  7


In [15]:
(tips
 .groupBy(tips.smoker)
 .count()
 .withColumn('percentage', 
    concat(round(col('count') / tips.count() * 100, 
    2), 
    lit(' %'))).show())

+------+-----+----------+
|smoker|count|percentage|
+------+-----+----------+
|    No|  151|   61.89 %|
|   Yes|   93|   38.11 %|
+------+-----+----------+



 b. Create a column that contains the tip percentage


In [16]:
(tips
 .withColumn('tip_percentage', 
    concat(round(tips.tip / tips.total_bill * 100, 
    2), 
    lit(' %'))).show())

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        5.94 %|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|       16.05 %|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|       16.66 %|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|       13.98 %|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|       14.68 %|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|       18.62 %|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|       22.81 %|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|       11.61 %|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|       13.03 %|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|       21.85 %|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|       16.65 %|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|       14.18 %|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|       1


 c. Calculate the average tip percentage for each combination of sex and smoker.

In [17]:
(tips
 .withColumn('tip_percentage', 
    col('tip') / col('total_bill') * 100)
 .groupBy('sex')
 .pivot('smoker')
 .agg(round(mean('tip_percentage'), 
    2)).show())

+------+-----+-----+
|   sex|   No|  Yes|
+------+-----+-----+
|Female|15.69|18.22|
|  Male|16.07|15.28|
+------+-----+-----+



4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

 - Convert the temperatures to fahrenheit.


In [18]:
from vega_datasets import data

weather = data.seattle_weather().assign(date = lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



In [19]:
(weather
 .select(round((weather.temp_max * (9/5) + 32), 
    2)
 .alias('temp_max_f'), 
    round((weather.temp_min * (9/5) + 32), 
    2)
 .alias('temp_min_f')).show(5))

+----------+----------+
|temp_max_f|temp_min_f|
+----------+----------+
|     55.04|      41.0|
|     51.08|     37.04|
|     53.06|     44.96|
|     53.96|     42.08|
|     48.02|     37.04|
+----------+----------+
only showing top 5 rows



 - Which month has the most rain, on average?
 

In [20]:
(weather
 .withColumn('month', 
    month('date'))
 .groupBy('month')
 .agg(round(avg('precipitation'), 
    2)
 .alias('average_precipitation'))
 .sort(desc('average_precipitation')).show())

+-----+---------------------+
|month|average_precipitation|
+-----+---------------------+
|   11|                 5.35|
|   12|                 5.02|
|    3|                 4.89|
|   10|                 4.06|
|    1|                 3.76|
|    2|                 3.73|
|    4|                 3.13|
|    9|                 1.96|
|    5|                 1.67|
|    8|                 1.32|
|    6|                 1.11|
|    7|                 0.39|
+-----+---------------------+



In [30]:
(weather
 .withColumn('month', 
    month('date'))
 .groupBy('month')
 .agg(round(sum('precipitation') / 4, 
    2)
 .alias('average_precipitation'))
 .sort(desc('average_precipitation')).show())

+-----+---------------------+
|month|average_precipitation|
+-----+---------------------+
|   11|               160.63|
|   12|               155.68|
|    3|               151.55|
|   10|               125.85|
|    1|                116.5|
|    2|                105.5|
|    4|                93.85|
|    9|                58.87|
|    5|                51.88|
|    8|                40.93|
|    6|                33.23|
|    7|                12.05|
+-----+---------------------+



- Which year was the windiest?


In [41]:
(weather
 .withColumn('year', 
    year('date'))
 .groupBy('year')
 .agg(round(avg('wind'), 
    2)
 .alias('average_wind_speed'))
 .sort(desc('average_wind_speed')).show())

+----+------------------+
|year|average_wind_speed|
+----+------------------+
|2012|               3.4|
|2014|              3.39|
|2015|              3.16|
|2013|              3.02|
+----+------------------+



In [42]:
(weather
 .withColumn('year', 
    year('date'))
 .groupBy('year')
 .agg(round(sum('wind'), 
    2)
 .alias('cumulative_wind_speed'))
 .sort(desc('cumulative_wind_speed')).show())

+----+---------------------+
|year|cumulative_wind_speed|
+----+---------------------+
|2012|               1244.7|
|2014|               1236.5|
|2015|               1153.3|
|2013|               1100.8|
+----+---------------------+



 - What is the most frequent type of weather in January?


In [22]:
(weather
 .withColumn('month', 
    month('date'))
 .filter(col('month') == 1)
 .groupBy('weather')
 .count()
 .sort(col('count')
 .desc()).show())

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



 - What is the average high and low temperature on sunny days in July in 2013 and 2014?
 

In [23]:
(weather
 .filter(month('date') == 7)
 .filter(year('date') > 2012)
 .filter(year('date') < 2015)
 .filter(col('weather') == lit('sun'))
 .agg(round(avg('temp_max'), 
    2).alias('average_max_temp'), 
    round(avg('temp_min'), 
    2).alias('average_min_temp')).show())

+----------------+----------------+
|average_max_temp|average_min_temp|
+----------------+----------------+
|           26.83|           14.18|
+----------------+----------------+



 - What percentage of days were rainy in q3 of 2015?
 

In [36]:
(weather
 .filter(year('date') == 2015)
 .filter(quarter('date') == 3)
 .select(when(col('weather') == 'rain', 
    1)
 .otherwise(0)
 .alias('rain'))
 .agg(round((avg('rain') * 100), 
    2)
 .alias('Rainy Day % in Q3 2015')).show())

+----------------------+
|Rainy Day % in Q3 2015|
+----------------------+
|                  2.17|
+----------------------+



- For each year, find what percentage of days it rained (had non-zero precipitation).

In [25]:
(weather
 .withColumn('year', 
    year('date'))
 .select(when(col('precipitation') > 0, 
    1).otherwise(0)
 .alias('rain'), 
    'year')
 .groupBy('year')
 .agg((round(avg('rain'), 
    4) * 100)
 .alias('Percentage of Rainy Days By Year')).show())


+----+--------------------------------+
|year|Percentage of Rainy Days By Year|
+----+--------------------------------+
|2012|                           48.36|
|2013|                           41.64|
|2014|              41.099999999999994|
|2015|                           39.45|
+----+--------------------------------+

